In [1]:
# Data Processing
import pandas as pd
from PIL import Image
from examples.circle_square.generator import generate_balanced_dataset, draw_shapes
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import mean_absolute_error

import torch
import torch.nn as nn
import torch.nn.functional as F

from ltn_imp.automation.knowledge_base import KnowledgeBase
from ltn_imp.automation.data_loaders import LoaderWrapper

## Data Processing and Visualization

In [2]:
data, metadata = generate_balanced_dataset(50)

In [3]:
data = pd.DataFrame(data)

In [4]:
metadata = pd.DataFrame(metadata)
metadata

,circle_center_x,circle_center_y,circle_radius,rect_tl_x,rect_tl_y,rect_br_x,rect_br_y
0,102,56,18,13,91,59,47
1,29,63,17,81,86,101,49
2,45,85,11,22,122,65,100
3,34,38,18,63,103,103,59
4,41,32,12,27,80,75,31
...,...,...,...,...,...,...,...
145,95,99,10,78,114,113,88
146,74,90,17,56,115,102,69
147,90,28,14,67,56,117,11
148,110,62,12,90,89,124,41


In [5]:
image_paths = [item for item in data[0]]
images = []

for path in image_paths:
    try:
        img = Image.open(path).convert('RGB')  # Convert to RGB to ensure consistency
        img = np.array(img)
        img_tensor = torch.tensor(img, dtype=torch.float32)  # Convert to PyTorch tensor
        images.append(img_tensor)
    except Exception as e:
        print(f"Error loading image {path}: {e}")

In [6]:
class EvalDataset(Dataset):
    def __init__(self, images, metadata):
        self.images = torch.stack([torch.tensor(image).permute(2, 0, 1) for image in images])
        self.metadata = torch.tensor(metadata).float()

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        metadata = self.metadata[idx]
        return image, metadata
    
batch_size = 25

circle_metadata = metadata[["circle_center_x","circle_center_y", "circle_radius"]]

rectangle_metadata = metadata[["rect_tl_x",	"rect_tl_y"	,"rect_br_x"	,"rect_br_y"]]

train_images, test_images, train_circle_labels, test_circle_labels = train_test_split( images, circle_metadata.values, test_size=0.2, random_state=42)

train_images, test_images, train_rect_labels, test_rect_labels = train_test_split(images, rectangle_metadata.values, test_size=0.2, random_state=42)

# Create the training and test datasets for circles
train_circle_dataset = EvalDataset(train_images, train_circle_labels)
test_circle_dataset = EvalDataset(test_images, test_circle_labels)

# Create the training and test datasets for rectangles
train_rect_dataset = EvalDataset(train_images, train_rect_labels)
test_rect_dataset = EvalDataset(test_images, test_rect_labels)

# Create the dataloaders
batch_size = 25

# Circle dataloaders
train_circle_dataloader = DataLoader(train_circle_dataset, batch_size=batch_size, shuffle=True)
test_circle_dataloader = DataLoader(test_circle_dataset, batch_size=batch_size, shuffle=False)

# Rectangle dataloaders
train_rect_dataloader = DataLoader(train_rect_dataset, batch_size=batch_size, shuffle=True)
test_rect_dataloader = DataLoader(test_rect_dataset, batch_size=batch_size, shuffle=False)

/var/folders/mv/_dg3pqgn2zdf7f95_1dg07rw0000gn/T/ipykernel_27484/445107885.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.images = torch.stack([torch.tensor(image).permute(2, 0, 1) for image in images])


# Models and Evaluation Functions

In [7]:
class CircleDetector(nn.Module):
    def __init__(self):
        super(CircleDetector, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(in_features=64*16*16, out_features=128)
        self.bn_fc1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(in_features=128, out_features=3)

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        x = F.leaky_relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = F.leaky_relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.leaky_relu(self.bn_fc1(self.fc1(x)))
        x = self.fc2(x)
        c_x, c_y, r = (x[:, 0]), (x[:, 1]), (x[:, 2])
        return c_x, c_y, r
class RectangleDetector(torch.nn.Module):
    def __init__(self):
        super(RectangleDetector, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(in_features=64*16*16, out_features=128)
        self.bn_fc1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(in_features=128, out_features=4)

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        x = F.leaky_relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = F.leaky_relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.leaky_relu(self.bn_fc1(self.fc1(x)))
        x = self.fc2(x)
        t_x, t_y,b_x, b_y = (x[:, 0]), (x[:, 1]), (x[:, 2]), (x[:, 3])
        return t_x, t_y, b_x, b_y

In [8]:
def train_circle(model, dataloader, criterion, optimizer, num_epochs=25, device='cpu'):
    model.to(device)
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            c_x, c_y, r = model(inputs)
            outputs = torch.stack((c_x, c_y, r), dim=1)
            
            # Calculate loss
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}')

def train_rect(model, dataloader, criterion, optimizer, num_epochs=25, device='cpu'):
    model.to(device)
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            tl_x, tl_y, br_x, br_y = model(inputs)
            outputs = torch.stack((tl_x, tl_y, br_x, br_y), dim=1)
            
            # Calculate loss
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}')


def evaluate_model_circle(model, dataloader, device='cpu'):
    model.to(device)
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            c_x, c_y, r = model(inputs)
            preds = torch.stack((c_x, c_y, r), dim=1)

            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    mae = mean_absolute_error(all_labels, all_preds)
    print(f'Mean Absolute Error: {mae:.4f}')
    return mae


def evaluate_model_rect(model, dataloader, device='cpu'):
    model.to(device)
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            tl_x, tl_y, br_x, br_y = model(inputs)
            preds = torch.stack((tl_x, tl_y, br_x, br_y), dim=1)

            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    mae = mean_absolute_error(all_labels, all_preds)
    print(f'Mean Absolute Error: {mae:.4f}')
    return mae


# Circle

# Rectangle

# LTN

In [9]:
class EvalDataset(Dataset):
    def __init__(self, images, metadata):
        self.images = torch.stack([torch.tensor(image).permute(2, 0, 1) for image in images])
        
        # Unpack the metadata into separate variables
        metadata_tensor = torch.tensor(metadata).float()
        self.t_x, self.t_y, self.b_x, self.b_y = metadata_tensor[:, 0], metadata_tensor[:, 1], metadata_tensor[:, 2], metadata_tensor[:, 3]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        t_x, t_y, b_x, b_y = self.t_x[idx], self.t_y[idx], self.b_x[idx], self.b_y[idx]
        return image, t_x, t_y, b_x, b_y

batch_size = 64

rectangle_metadata = metadata[["rect_tl_x", "rect_tl_y", "rect_br_x", "rect_br_y"]]
train_images, test_images, train_rect_labels, test_rect_labels = train_test_split(images, rectangle_metadata.values, test_size=0.2, random_state=42)

# Create the training and test datasets for rectangles
train_rect_dataset = EvalDataset(train_images, train_rect_labels)
test_rect_dataset = EvalDataset(test_images, test_rect_labels)

train_rect_dataloader = DataLoader(train_rect_dataset, batch_size=batch_size, shuffle=True)
test_rect_dataloader = DataLoader(test_rect_dataset, batch_size=batch_size, shuffle=False)

/var/folders/mv/_dg3pqgn2zdf7f95_1dg07rw0000gn/T/ipykernel_27484/1125484005.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.images = torch.stack([torch.tensor(image).permute(2, 0, 1) for image in images])


In [10]:
def evaluate_model_rect(model, dataloader, device='cpu'):
    model.to(device)
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, t_x, t_y, b_x, b_y in dataloader:
            inputs = inputs.to(device)
            t_x, t_y, b_x, b_y = t_x.to(device), t_y.to(device), b_x.to(device), b_y.to(device)
            tl_x, tl_y, br_x, br_y = model(inputs)
            preds = torch.stack((tl_x, tl_y, br_x, br_y), dim=1)
            labels = torch.stack((t_x, t_y, b_x, b_y), dim=1)

            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    mae = mean_absolute_error(all_labels, all_preds)
    print(f'Mean Absolute Error: {mae:.4f}')
    return mae

In [11]:
rect = RectangleDetector()

expression_1 = "all i. (Rect(i, t1, t2, b1, b2))"
expression_2 = "all t1. ((t1 = t1) and (b1 = b11) and (t2 = t22) and (b2 = b22))"

learning_rules = [expression_1, expression_2]

loader = LoaderWrapper(loader=train_rect_dataloader, variables=["i"], targets=["t11", "t22", "b11", "b22"])

rule_to_data_loader_mapping = {
    expression_1: [loader],
    expression_2: [loader], 
}

predicates = {
    "Rect": rect
}

quantifier_impls = { "forall" : "pmean_error" }


kb = KnowledgeBase(learning_rules = learning_rules, ancillary_rules=[], rule_to_data_loader_mapping=rule_to_data_loader_mapping, 
                   predicates=predicates, quantifier_impls=quantifier_impls)

In [12]:
evaluate_model_rect(rect, train_rect_dataloader, device='cpu')
print()
kb.optimize(num_epochs=5, lr=0.001, log_steps=1)
print()
evaluate_model_rect(rect, test_rect_dataloader, device='cpu')

Mean Absolute Error: 72.4398

['∀ i.(Rect(i, t1, t2, b1, b2))', '∀ t1.(((t1 = t1) & ((b1 = b11) & ((t2 = t22) & (b2 = b22)))))']
Rule Outputs:  [tensor(1., grad_fn=<RsubBackward1>), tensor(nan, grad_fn=<RsubBackward1>)]
Epoch 1/5, Loss: nan

['∀ i.(Rect(i, t1, t2, b1, b2))', '∀ t1.(((t1 = t1) & ((b1 = b11) & ((t2 = t22) & (b2 = b22)))))']
Rule Outputs:  [tensor(1., grad_fn=<RsubBackward1>), tensor(nan, grad_fn=<RsubBackward1>)]
Epoch 2/5, Loss: nan

['∀ i.(Rect(i, t1, t2, b1, b2))', '∀ t1.(((t1 = t1) & ((b1 = b11) & ((t2 = t22) & (b2 = b22)))))']
Rule Outputs:  [tensor(1., grad_fn=<RsubBackward1>), tensor(nan, grad_fn=<RsubBackward1>)]
Epoch 3/5, Loss: nan

['∀ i.(Rect(i, t1, t2, b1, b2))', '∀ t1.(((t1 = t1) & ((b1 = b11) & ((t2 = t22) & (b2 = b22)))))']
Rule Outputs:  [tensor(1., grad_fn=<RsubBackward1>), tensor(nan, grad_fn=<RsubBackward1>)]
Epoch 4/5, Loss: nan

['∀ i.(Rect(i, t1, t2, b1, b2))', '∀ t1.(((t1 = t1) & ((b1 = b11) & ((t2 = t22) & (b2 = b22)))))']
Rule Outputs:  [tensor(

ValueError: Input contains NaN.